# Klasyfikacja bankructwa na podstawie danych ze zbioru "Company Bankruptcy Prediction"

Julia Kaznowska, Piotr Wilczyński <br>
23/04/2022 <br>
Politechnika Warszawska, Wydział Matematyki i Nauk Informacyjnych, Wstęp do uczenia maszynowego

## Import niezbędnych bibliotek oraz zbioru danych

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# zbior danych
df = pd.read_csv("data.csv")

# wyświetlanie wizualizacji
%matplotlib inline

## Wstępne informacje o danych

In [6]:
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [7]:
df.shape

(6819, 96)

Dane mają 96 kolumn i 6819 rekordów.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 6819 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2    ROA(A) before interest and % after tax                   6819 non-null   float64
 3    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4    Operating Gross Margin                                   6819 non-null   float64
 5    Realized Sales Gross Margin                              6819 non-null   float64
 6    Operating Profit Rate                                    6819 non-null   float64
 7    Pre-tax net Interest Rate                                6819 non-null   float64
 8    After-tax net Int

Nie ma oczywistych braków danych, ale widzimy dwie wyróżniające się wartości: Net Income Flag i Liability-Assets Flag. Obie w przeciwieństwie do wszystkich innych zmiennych opisujących są całkowite a nie zmiennoprzecinkowe. Zobaczmy ich wartości.

In [9]:
unique, counts = np.unique(df[" Liability-Assets Flag"], return_counts=True)
unique, counts

(array([0, 1], dtype=int64), array([6811,    8], dtype=int64))

In [10]:
unique, counts = np.unique(df[" Net Income Flag"], return_counts=True)
unique, counts

(array([1], dtype=int64), array([6819], dtype=int64))

Zmienna "Net Income Flag" zawsze ma wartość 1, zatem nie ma wpływu na predykcyjność. Możemy się jej pozbyć. Zmienna "Liability-Assets Flag" ma wartość 1 tylko w 8 przypadkach, zobaczmy jakich.

In [11]:
df.loc[df[" Liability-Assets Flag"] == 1]

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
56,1,0.066933,0.057185,0.054821,0.601861,0.601861,0.998825,0.796779,0.808717,0.302760,...,0.525651,5.803037e-03,0.623648,0.601857,1.000000,0.182790,0.026763,0.565021,1,0.009178
1869,1,0.392775,0.432239,0.432946,0.586921,0.586921,0.998776,0.797126,0.809068,0.303470,...,0.722761,2.417803e-03,0.622734,0.586923,0.976180,0.000000,0.026703,0.564698,1,0.009879
1870,1,0.277726,0.314708,0.307351,0.596621,0.596650,0.998976,0.797176,0.809113,0.303138,...,0.664814,3.231135e-03,0.623270,0.596619,0.902744,0.199162,0.026755,0.564978,1,0.008950
2001,1,0.438795,0.090166,0.464586,0.540776,0.540776,0.997789,0.790787,0.802967,0.294457,...,0.411809,1.109791e-02,0.625487,0.540775,0.996912,0.209222,0.026779,0.565098,1,0.008753
2470,1,0.404036,0.223615,0.430055,0.586611,0.586611,0.998568,0.796179,0.808154,0.302249,...,0.572881,8.658754e-03,0.623173,0.586607,0.916329,0.218785,0.026745,0.564930,1,0.008500
2735,0,0.436894,0.453718,0.479522,0.585062,0.585062,0.998495,0.796770,0.808785,0.303434,...,0.747290,4.202211e-04,0.557613,0.585059,0.885473,0.133503,0.026744,0.564922,1,0.009546
6613,0,0.279676,0.283362,0.303014,0.637520,0.637520,0.998785,0.797055,0.809000,0.303325,...,0.705559,3.030000e+09,0.623292,0.637516,0.841826,0.265220,0.026791,0.565158,1,0.000000
6640,1,0.196802,0.211023,0.221425,0.598056,0.598056,0.998933,0.796144,0.808149,0.301423,...,0.519388,1.758765e-02,0.623465,0.598051,0.856906,0.259280,0.026769,0.565052,1,0.003946


Flaga Liability-Assets jest ustawiona na 1 zarówno kiedy bakrupt jest 1 jak i 0. Przy eliminacji outlierów i tak zamienimy flagę 1 na 0 (bo jest ich mniej niż 2.5%). Zmienna ta również nie będzie miała wartości predykcyjnej. Można ją zatem usunąć.

In [12]:
df = df.drop(" Liability-Assets Flag", axis = 1)
df = df.drop(" Net Income Flag", axis = 1)

In [13]:
100*df.loc[df["Bankrupt?"] == 1].shape[0]/df.shape[0] 

3.2262795131250916

Zaledwie lekko ponad 3% rekordów jest oznaczone flagą bankurpt.

## Podział zbioru danych na dane budujące i do walidacji

In [14]:
y = np.array(df["Bankrupt?"])
X = df.drop(["Bankrupt?"], axis = 1)
X_build, X_val, y_build, y_val = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=321
)
X_train, X_test, y_train, y_test = train_test_split(
    X_build, y_build, stratify=y_build, test_size=0.3, random_state=123
)

Eksport danych do walidacji

In [15]:
df_val = X_val.copy()
df_val["Bankrupt?"] = y_val.copy()
df_val.to_csv("data_val.csv")

## Prosty pre-processing danych

In [16]:
df_train = X_train.copy()
df_train["Bankrupt?"] = y_train.copy()
df_test = X_test.copy()
df_test["Bankrupt?"] = y_test.copy()

###  Outliery

Wartości poniżej precentyla 2.5 i powyżej 97.5 będziemy zastępować wartościami skrajnymi.

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

class outliers(BaseEstimator, TransformerMixin):
    def fit(self, X):
        self.upper_lim={}
        self.lower_lim={}
        for col in X.columns:
            self.upper_lim[col] = X[col].quantile(.975)
            self.lower_lim[col] = X[col].quantile(.025)
        return self
    def transform(self, X):
        y_temp = X["Bankrupt?"]
        X = X.drop("Bankrupt?", axis = 1)
        for col in X.columns:
            X[col] = np.where(X[col] < self.upper_lim[col], X[col], self.upper_lim[col])
            X[col] = np.where(X[col] > self.lower_lim[col], X[col], self.lower_lim[col])
        X["Bankrupt?"] = y_temp
        return X

### Zmiana kierunku korelacji

Użyjemy korelacji Spearmana. Wszystkie zmienne przekształcimy tak, aby były dodatnio skorelowane ze zmienną celu.

In [18]:
from scipy.stats import spearmanr

class direction_change(BaseEstimator, TransformerMixin):
    def fit(self, X):
        self.dir_change = set()
        y_temp = X["Bankrupt?"]
        X = X.drop("Bankrupt?", axis = 1)
        for col in X:
            for col in X.columns:
                if (spearmanr(X[col], y_temp)[0] < 0):
                    self.dir_change.add(col)
        X.insert(0, "Bankrupt?",y_temp)
        return self
    def transform(self, X):
        y_temp = X["Bankrupt?"]
        X = X.drop("Bankrupt?", axis = 1)
        for col in X:
            for col in X.columns:
                if (col in self.dir_change):
                    X[col] = -X[col]
        X.insert(0, "Bankrupt?",y_temp)
        return X

### Normalizacja min-max i pipeline

Aby znormalizować wartości użyjemy funkcji MinMaxScaler z biblioteki sklearn.

Wprowadzamy wszystkie przekształcenia do pipelina.

In [19]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('outliers', outliers()),
    ('direction_change', direction_change()),
    ('minmax', MinMaxScaler())
])

In [20]:
df_train = pd.DataFrame(pipe.fit_transform(df_train), columns = df.columns)
df_train["Bankrupt?"] = np.int64(df_train["Bankrupt?"])
df_test = pd.DataFrame(pipe.fit_transform(df_test), columns = df.columns)
df_test["Bankrupt?"] = np.int64(df_test["Bankrupt?"])

In [21]:
y_train = np.array(df_train["Bankrupt?"])
X_train = df_train.drop(["Bankrupt?"], axis = 1)

y_test = np.array(df_test["Bankrupt?"])
X_test = df_test.drop(["Bankrupt?"], axis = 1)

## Uczenie klasyfikatorów

In [60]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

### Support Vector Machines

In [51]:
param_grid_svm  = {
    'C' : [10, 100, 1000],
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto', 1, 0.2, 0.04],
    'class_weight' : ['balanced', None]
}

In [52]:
svm = SVC()
svm_tuned = GridSearchCV(svm, param_grid_svm, cv = 5, n_jobs = -1, scoring = 'roc_auc')
svm_tuned.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [10, 100, 1000],
                         'class_weight': ['balanced', None],
                         'gamma': ['scale', 'auto', 1, 0.2, 0.04],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='roc_auc')

In [53]:
svm_tuned.best_params_

{'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'poly'}

Wszystkie parametry wydają się być niezwykle istotne. Najlepszy okazał się klasyfikator z parametrem regularyzacji 10, automatycznie dobraną gammą (czyli w sklearnie 1/liczba_cech), jądrem wielomianowym i ze zbalansowanymi wagami.

In [54]:
svm_best = svm_tuned.best_estimator_

### Random Forest Classifier

In [87]:
param_grid_rfc  = {
    'n_estimators': np.arange(500, 3000),
    "max_depth": np.arange(1,93),
    "max_features": ['auto', 'log2'],
    "min_samples_split" : np.arange(2,10),
    "min_samples_leaf": np.arange(1,10),
    'bootstrap': [True, False]
}

In [88]:
rfc = RandomForestClassifier(random_state = 42)
rfc_tuned = RandomizedSearchCV(rfc, param_grid_rfc, cv = 5, n_jobs = -1, scoring = 'roc_auc', n_iter = 50, random_state = 42)
rfc_tuned.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92]),
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([2, 3, 4, 5, 6, 7, 8, 9]),
                     

In [89]:
rfc_tuned.best_params_

{'n_estimators': 2288,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 40,
 'bootstrap': True}

Najlepszy okazał się klasyfikator, który tworzy 2288 drzew, z minimalnie dwiema obserwacjami potrzebnymi do podzielenia węzła, minimalnie dwoma obserwacjami w liściu, z automatyczną maksymalną liczbą cech (czyli w sklearn pierwiastek z liczby cech), z maksymalną głębokością drzewa 40 i używający bootstrapu. 

In [90]:
rfc_best = rfc_tuned.best_estimator_

### Decision Tree Classifier

In [80]:
param_grid_dtc = {
    "max_depth": np.arange(1,93),
    "max_features": ['auto', 'sqrt', 'log2'],
    "min_samples_leaf": np.arange(1,10),
    "criterion": ["gini", "entropy"],
    "min_samples_split" : np.arange(2,10)
}

In [81]:
dtc = DecisionTreeClassifier(random_state = 42)
dtc_tuned = RandomizedSearchCV(dtc, param_grid_dtc, cv = 5, n_jobs = -1, scoring = 'roc_auc', n_iter = 1000, random_state = 42)
dtc_tuned.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=1000, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92]),
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([2, 3, 4, 5, 6, 7, 8, 9])},
            

In [82]:
dtc_tuned.best_params_

{'min_samples_split': 6,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 4,
 'criterion': 'entropy'}

In [83]:
dtc_best = dtc_tuned.best_estimator_

Najlepszy okazał się klasyfikator o maksymalnej glębokości 4, z minimalnie sześcioma obserwacjami potrzebnymi do podzielenia węzła, minimalnie jedną obserwacją w liściu, z automatyczną maksymalną liczbą cech (pierwiastek z liczby cech). Najlepiej jakość podziału mierzyć za pomocą entropii.

## Ocena jakości klasyfikatorów

Dokonujemy oceny jakości klasyfikatorów. Zdecydowaliśmy się na scoringi `accuracy`, `precision` oraz `recall`.

In [91]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def print_scores(s1, s2, s3):
    print(f"accuracy: mean = {np.round(np.mean(s1), 4)}, std = {np.round(np.std(s1), 4)}")
    print(f"precision: mean = {np.round(np.mean(s2), 4)}, std = {np.round(np.std(s2), 4)}")
    print(f"recall: mean = {np.round(np.mean(s3), 4)}, std = {np.round(np.std(s3), 4)}")
    
def print_scores_test(s1, s2, s3):
    print("accuracy:", np.round(s1, 4))
    print("precision:", np.round(s2, 4))
    print("recall:", np.round(s3, 4))

### Support Vector Machines

In [92]:
train_acc = cross_val_score(svm_best, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(svm_best, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(svm_best, X_train, y_train, scoring='recall', cv = 4)

In [93]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.9054, std = 0.005
precision: mean = 0.2246, std = 0.0238
recall: mean = 0.787, std = 0.0921


In [94]:
test_acc = accuracy_score(y_test, svm_best.predict(X_test))
test_pr = precision_score(y_test, svm_best.predict(X_test))
test_rec = recall_score(y_test, svm_best.predict(X_test))

In [95]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9064
precision: 0.2143
recall: 0.7174


Wyniki na danych testowych i treningowych są mniej więcej takie same. 

### Random Forest Classifier

In [96]:
train_acc = cross_val_score(rfc_best, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(rfc_best, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(rfc_best, X_train, y_train, scoring='recall', cv = 4)

In [97]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.9713, std = 0.0
precision: mean = 0.7071, std = 0.0711
recall: mean = 0.213, std = 0.0711


In [98]:
test_acc = accuracy_score(y_test, rfc_best.predict(X_test))
test_pr = precision_score(y_test, rfc_best.predict(X_test))
test_rec = recall_score(y_test, rfc_best.predict(X_test))

In [99]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9679
precision: 0.5
recall: 0.2174


Na danych treningowych otrzymujemy znacząco lepsze 'accuracy' i 'precision', może to świadczyć o tym, że model cały czas jest jeszcze przetrenowany.

### Decision Tree Classifier

In [100]:
train_acc = cross_val_score(dtc_best, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(dtc_best, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(dtc_best, X_train, y_train, scoring='recall', cv = 4)

In [101]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.9626, std = 0.0041
precision: mean = 0.1518, std = 0.1523
recall: mean = 0.1019, std = 0.137


In [102]:
test_acc = accuracy_score(y_test, dtc_best.predict(X_test))
test_pr = precision_score(y_test, dtc_best.predict(X_test))
test_rec = recall_score(y_test, dtc_best.predict(X_test))

In [103]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9616
precision: 0.3448
recall: 0.2174


Wyniki dla danych testowych są nawet lepsze od wyników na danych treningowych.